# Hyperparamter Tuner for CNNs
This file is part of the bachelor thesis of Jad Dayoub, 7425569.

By setting values for different hyperparameters, a grid-search is done to find the optimal combination.

In [ ]:
import itertools
import numpy as np
import time
import torch.nn as nn
import torch.optim as optim

from models import TuneNet
from utils import fit, decorate_text
from synthetic_data_generation import generate_train_test_valid_data

In [1]:
def get_all_configurations(config):
    """
    Creates all possible combinations.
    """

    keys = config.keys()
    values = config.values()

    combinations = list(itertools.product(*values))

    configurations = [dict(zip(keys, combo)) for combo in combinations]

    return configurations


def parameter_tuning(data, labels, config, epochs=10, learn_plot=False):
    """ 
    Finds the optimal combination by testing all possible combinations.
    """

    dim = len(data[0][0])

    configurations = get_all_configurations(config)

    best_config = None 
    best_test_acc = 0

    count = 1
    total = len(configurations)
    start_time = time.time()
    for configuration in configurations:
        c1 = configuration["c1"]
        c2 = configuration["c2"]
        c3 = configuration.get("c3", None)
        fc = configuration["fc"]
        f_size = configuration["f_size"]
        lr = configuration["lr"]
        bs = configuration["bs"]

        train_data, test_data = generate_train_test_valid_data(data, labels, False, bs, 0.2)

        decorate_text(f"{count}/{total} | c1: {c1}, c2: {c2}"+(f", c3: {c3}" if c3 is not None else "")+f", fc: {fc}, f_size: {f_size}, lr: {lr}, bs: {bs}") 
        model = TuneNet(dim, c1, c2, c3, fc, f_size)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fun = nn.CrossEntropyLoss()
        _, _, _, test_acc, _ = fit(model, epochs, optimizer, loss_fun, train_data, test_data, learn_plot=False)

        if test_acc[-1] > best_test_acc:
            best_test_acc = test_acc[-1]
            best_config = configuration
        
        count += 1
    
    end_time = time.time()
    duration = end_time - start_time
    duration_str = time.strftime("%H:%M:%S", time.gmtime(duration))
    
    print(f"\nBest configuration with Test Accuracy of {best_test_acc}: (Runtime: {duration_str})")
    for key, value in best_config.items():
        print(f"{key}: {value}")

In [ ]:
config = {
    "c1": [16, 32],
    "c2": [32, 64],
    "c3": [None, 64],
    "fc": [128, 256],
    "f_size": [3],
    "lr": [0.001, 0.0001],
    "bs": [8, 16, 32]
}

dataset = np.load("../data/256x256_ml_ppa_data_binary.npz")
labels = dataset["labels"]

i = 0
print()
print(f"testing on {dataset.files[i]}")
print()
for j in range(3):
    data = dataset[dataset.files[i]]
    parameter_tuning(data, labels, config, learn_plot=False)